# Clean data

Data scraped from https://esmvaltool.cloud.dkrz.de/shared/esmvaltool/climate4impact/work/bias_and_change/visualize/bias_vs_change.html

copy/pasted html table to excel and saved as csv.

TODO: make recipe spit out correct output (and altair spec?)

In [2]:
import pandas as pd

In [184]:
df = (
    pd.read_csv("scraped_html.csv", header=[0, 1], index_col=0)
    .reset_index()
    .set_index("dataset")
)
df = pd.DataFrame(df.to_records())
df.columns = ["dataset", "tas_bias", "pr_bias", "tas_change", "pr_change"]
df["project"] = df.apply(lambda row: row.dataset.split("_")[0], axis=1)
df["member"] = df.apply(lambda row: row.dataset.split("_")[-1], axis=1)
df["model"] = df.apply(lambda row: row.dataset.split("_")[1], axis=1)

# convert precip to mm/day
df["pr_bias"] *= 86400
df["pr_change"] *= 86400

df.to_csv("../static/data/recipe_output.csv", index=False)
df.to_json("../static/data/recipe_output.json", orient="records")
df

,dataset,tas_bias,pr_bias,tas_change,pr_change,project,member,model
0,CMIP5_ACCESS1-0_r1i1p1,3.19,1.69344,2.36,0.000691,CMIP5,r1i1p1,ACCESS1-0
1,CMIP5_ACCESS1-3_r1i1p1,3.11,1.66752,2.24,0.054173,CMIP5,r1i1p1,ACCESS1-3
2,CMIP5_BNU-ESM_r1i1p1,4.08,1.61568,2.44,0.002557,CMIP5,r1i1p1,BNU-ESM
3,CMIP5_CCSM4_r1i1p1,3.12,1.46016,1.95,-0.016589,CMIP5,r1i1p1,CCSM4
4,CMIP5_CCSM4_r2i1p1,3.23,1.49472,1.65,0.005184,CMIP5,r2i1p1,CCSM4
...,...,...,...,...,...,...,...,...
380,CMIP6_UKESM1-0-LL_r1i1p1f2,3.51,1.52064,3.76,0.019267,CMIP6,r1i1p1f2,UKESM1-0-LL
381,CMIP6_UKESM1-0-LL_r2i1p1f2,3.69,1.48608,3.92,0.018662,CMIP6,r2i1p1f2,UKESM1-0-LL
382,CMIP6_UKESM1-0-LL_r3i1p1f2,3.36,1.50336,3.48,0.034819,CMIP6,r3i1p1f2,UKESM1-0-LL
383,CMIP6_UKESM1-0-LL_r4i1p1f2,3.17,1.51200,3.30,0.003707,CMIP6,r4i1p1f2,UKESM1-0-LL


# Plot

Adapted from:
- https://altair-viz.github.io/gallery/scatter_linked_brush.html

In [76]:
import altair as alt

In [200]:
def plot(source):
    brush = alt.selection(type="interval", resolve="global")

    input_dropdown = alt.binding_select(options=["CMIP5", "CMIP6"])
    dropdown_selection = alt.selection_single(
        fields=["project"], bind=input_dropdown, name="Select"
    )

    base = (
        alt.Chart(source)
        .mark_circle(size=150)
        .encode(
            color=alt.condition(
                brush, "model:N", alt.ColorValue("lightgray"), legend=None
            ),
            tooltip=["dataset:N"],
        )
        .transform_filter(dropdown_selection)
        .add_selection(brush, dropdown_selection)
    )

    tas = base.encode(
        x=alt.X(
            "tas_bias:Q", axis=alt.Axis(title="Bias with respect to ERA5 (1981-2010)")
        ),
        y=alt.Y(
            "tas_change:Q", axis=alt.Axis(title="Projected change (2050 versus 2000)")
        ),
    ).properties(title="Temperature (K)")

    pr = base.encode(
        x=alt.X(
            "pr_bias:Q", axis=alt.Axis(title="Bias with respect to ERA5 (1981-2010)")
        ),
        y=alt.Y(
            "pr_change:Q", axis=alt.Axis(title="Projected change (2050 versus 2000)")
        ),
    ).properties(title="Precipitation (mm/day)")

    return tas | pr


plot(df)

alt.HConcatChart(...)

In [196]:
# For the frontend, don't use live data but load from URL
chart = plot("/data/recipe_output.csv")
chart.save("../static/specs/default.json")

# Now, this spec can be used in pages/index.vue as "/specs/default.json"

# Inspect the vega-lite spec
print(chart.to_json())

{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json",
  "config": {
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 400
    }
  },
  "data": {
    "url": "/data/recipe_output.csv"
  },
  "hconcat": [
    {
      "encoding": {
        "color": {
          "condition": {
            "field": "model",
            "legend": null,
            "selection": "selector103",
            "type": "nominal"
          },
          "value": "lightgray"
        },
        "tooltip": [
          {
            "field": "dataset",
            "type": "nominal"
          }
        ],
        "x": {
          "field": "tas_bias",
          "type": "quantitative"
        },
        "y": {
          "field": "tas_change",
          "type": "quantitative"
        }
      },
      "mark": {
        "size": 75,
        "type": "circle"
      },
      "selection": {
        "Select": {
          "bind": {
            "input": "select",
            "options": [
              